In [ ]:
import pandas as pd
import os
import googlemaps
from dotenv import load_dotenv
import time

df = pd.read_csv('../data/locales_procesado.csv')

In [ ]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
gmaps = googlemaps.Client(key=google_api_key)

In [ ]:
# Agregar nuevas columnas al dataframe
df['competencia_count'] = 0
df['competencia_rating'] = 0.0

for index, row in df.iterrows():
    latitude = row['latitud']
    longitude = row['longitud']
    categoria_negocio = row['tipo_negocio']

    try:
        places_result = gmaps.places_nearby(
            location=(latitude, longitude),
            radius=500,
            type=categoria_negocio
        )

        count = len(places_result['results'])
        ratings = [place.get('rating') for place in places_result['results'] if 'rating' in place]
        avg_rating = sum(ratings) / len(ratings) if ratings else 0.0

        df.at[index, 'competencia_count'] = count
        df.at[index, 'competencia_rating'] = avg_rating

        time.sleep(1)  # Evitar superar el límite de peticiones

    except Exception as e:
        print(f"Error en la fila {index}: {e}")

In [ ]:
df.to_csv('locales_enriquecido.csv', index=False)